In [1]:
1+1

2

In [2]:
import os
import glob


def find_pdf_files(search_name="더존비즈온"):
    """PDF 파일 검색"""
    print(f"'{search_name}'이 포함된 PDF 파일 검색 중...")

    # 현재 디렉토리와 하위 디렉토리에서 검색
    for root, dirs, files in os.walk("."):
        for file in files:
            if file.endswith(".pdf") and search_name in file:
                full_path = os.path.join(root, file)
                print(f"발견: {full_path}")
                return full_path

    print("해당 파일을 찾을 수 없습니다.")
    return None


# 파일 검색 및 처리
found_file = find_pdf_files()


'더존비즈온'이 포함된 PDF 파일 검색 중...
발견: .\20250630_더존비즈온.pdf


In [7]:
import PyPDF2
import pandas as pd
import re
from typing import List, Dict
import json


class PDFToKnowledgeConverter:
    def __init__(self):
        self.knowledge_base = []

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """PDF에서 텍스트 추출"""
        text = ""
        with open(pdf_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text

    def extract_tables_from_pdf(self, pdf_path: str) -> List[pd.DataFrame]:
        """PDF에서 표 추출 (tabula-py 사용)"""
        try:
            import tabula

            tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)
            return tables
        except ImportError:
            print("tabula-py가 설치되지 않았습니다. pip install tabula-py")
            return []

    def clean_text(self, text: str) -> str:
        """텍스트 정제"""
        # 불필요한 공백, 특수문자 제거
        text = re.sub(r"\s+", " ", text)
        text = re.sub(r"[^\w\s가-힣.,?!]", " ", text)
        return text.strip()

    def split_into_chunks(self, text: str, chunk_size: int = 500) -> List[str]:
        """텍스트를 청크로 분할"""
        words = text.split()
        chunks = []
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            chunks.append(chunk)
        return chunks

    def table_to_qa_pairs(self, table: pd.DataFrame) -> List[Dict]:
        """표를 Q&A 쌍으로 변환"""
        qa_pairs = []

        # 각 행을 질문-답변으로 변환
        for index, row in table.iterrows():
            for col in table.columns:
                if pd.notna(row[col]):
                    question = f"{col}에 대해 알려주세요."
                    answer = f"{col}은(는) {row[col]}입니다."

                    qa_pairs.append(
                        {
                            "question": question,
                            "answer": answer,
                            "source": "table",
                            "metadata": {"table_row": index, "column": col},
                        }
                    )

        return qa_pairs

    def text_to_knowledge_structure(self, text_chunks: List[str]) -> List[Dict]:
        """텍스트 청크를 지식 구조로 변환"""
        knowledge_items = []

        for i, chunk in enumerate(text_chunks):
            # 간단한 제목 추출 (첫 문장을 제목으로 사용)
            sentences = chunk.split(".")
            title = (
                sentences[0][:50] + "..." if len(sentences[0]) > 50 else sentences[0]
            )

            knowledge_item = {
                "id": f"doc_chunk_{i}",
                "title": title,
                "content": chunk,
                "type": "document",
                "metadata": {"chunk_index": i, "word_count": len(chunk.split())},
            }
            knowledge_items.append(knowledge_item)

        return knowledge_items

    def process_pdf(self, pdf_path: str) -> Dict:
        """PDF 전체 처리 파이프라인"""
        result = {"document_knowledge": [], "table_qa_pairs": [], "summary": {}}

        # 1. 텍스트 추출 및 처리
        print("텍스트 추출 중...")
        raw_text = self.extract_text_from_pdf(pdf_path)
        cleaned_text = self.clean_text(raw_text)
        text_chunks = self.split_into_chunks(cleaned_text)

        # 2. 텍스트를 지식 구조로 변환
        print("텍스트 지식 구조 생성 중...")
        result["document_knowledge"] = self.text_to_knowledge_structure(text_chunks)

        # 3. 표 추출 및 Q&A 쌍 생성
        print("표 추출 및 Q&A 생성 중...")
        tables = self.extract_tables_from_pdf(pdf_path)

        for table_idx, table in enumerate(tables):
            if not table.empty:
                qa_pairs = self.table_to_qa_pairs(table)
                for qa in qa_pairs:
                    qa["metadata"]["table_index"] = table_idx
                result["table_qa_pairs"].extend(qa_pairs)

        # 4. 요약 정보
        result["summary"] = {
            "total_text_chunks": len(text_chunks),
            "total_tables": len(tables),
            "total_qa_pairs": len(result["table_qa_pairs"]),
            "total_knowledge_items": len(result["document_knowledge"]),
        }

        return result

    def save_knowledge_base(self, knowledge_data: Dict, output_path: str):
        """지식 베이스를 JSON 파일로 저장"""
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(knowledge_data, f, ensure_ascii=False, indent=2)
        print(f"지식 베이스가 {output_path}에 저장되었습니다.")


# 사용 예시
if __name__ == "__main__":
    # 필요한 패키지 설치
    # pip install PyPDF2 pandas tabula-py

    converter = PDFToKnowledgeConverter()

    # PDF 파일 경로
    pdf_file = ".\\20250630_더존비즈온.pdf"

    try:
        # PDF 처리
        knowledge_data = converter.process_pdf(pdf_file)

        # 결과 출력
        print("\n=== 처리 결과 요약 ===")
        print(f"텍스트 청크 수: {knowledge_data['summary']['total_text_chunks']}")
        print(f"추출된 표 수: {knowledge_data['summary']['total_tables']}")
        print(f"생성된 Q&A 쌍 수: {knowledge_data['summary']['total_qa_pairs']}")

        # 지식 베이스 저장
        converter.save_knowledge_base(knowledge_data, "knowledge_base.json")

        # 샘플 출력
        if knowledge_data["document_knowledge"]:
            print("\n=== 문서 지식 샘플 ===")
            print(knowledge_data["document_knowledge"][0])

        if knowledge_data["table_qa_pairs"]:
            print("\n=== 표 Q&A 샘플 ===")
            print(knowledge_data["table_qa_pairs"][0])

    except Exception as e:
        print(f"처리 중 오류 발생: {e}")

텍스트 추출 중...
텍스트 지식 구조 생성 중...
표 추출 및 Q&A 생성 중...
처리 중 오류 발생: 'utf-8' codec can't decode byte 0xba in position 211: invalid start byte


In [6]:
import PyPDF2
import pandas as pd
import re
from typing import List, Dict
import json


class PDFToKnowledgeConverter:
    def __init__(self):
        self.knowledge_base = []

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """PDF에서 텍스트 추출 (인코딩 문제 해결)"""
        text = ""
        try:
            # 바이너리 모드로 파일 열기
            with open(pdf_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
        except Exception as e:
            print(f"PDF 텍스트 추출 오류: {e}")
        return text

    def extract_tables_from_pdf(self, pdf_path: str) -> List[pd.DataFrame]:
        """PDF에서 표 추출 (인코딩 옵션 추가)"""
        try:
            import tabula

            # 인코딩 옵션을 여러 개 시도
            encodings = ["utf-8", "cp949", "euc-kr", "cp1252", "iso-8859-1"]

            for encoding in encodings:
                try:
                    print(f"인코딩 {encoding} 시도 중...")
                    tables = tabula.read_pdf(
                        pdf_path, pages="all", multiple_tables=True, encoding=encoding
                    )
                    print(f"성공: {encoding} 인코딩으로 표 추출 완료")
                    return tables
                except UnicodeDecodeError:
                    continue

            print("모든 인코딩 방식 실패, 빈 리스트 반환")
            return []

        except ImportError:
            print("tabula-py가 설치되지 않았습니다.")
            print("다음 명령어로 설치하세요:")
            print("pip install tabula-py")
            print("pip install jpype1")
            return []
        except Exception as e:
            print(f"표 추출 오류: {e}")
            return []

    def clean_text(self, text: str) -> str:
        """텍스트 정제 (한글 지원)"""
        if not text:
            return ""

        # 불필요한 공백 정리
        text = re.sub(r"\s+", " ", text)

        # 특수문자 제거하되 한글, 영문, 숫자, 기본 문장부호 유지
        text = re.sub(r"[^\w\s가-힣.,?!():\-]", " ", text)
        return text.strip()

    # ... 나머지 메서드들은 동일 ...


# 안전한 실행 함수
def safe_process_pdf(pdf_path: str):
    """안전한 PDF 처리 함수"""
    import os

    if not os.path.exists(pdf_path):
        print(f"파일을 찾을 수 없습니다: {pdf_path}")
        return None

    try:
        converter = PDFToKnowledgeConverter()

        print("=== PDF 처리 시작 ===")
        knowledge_data = converter.process_pdf(pdf_path)

        print("\n=== 처리 결과 ===")
        print(f"텍스트 청크: {knowledge_data['summary']['total_text_chunks']}개")
        print(f"추출된 표: {knowledge_data['summary']['total_tables']}개")
        print(f"Q&A 쌍: {knowledge_data['summary']['total_qa_pairs']}개")

        # 결과 저장
        output_file = "buzzon_knowledge_base.json"
        converter.save_knowledge_base(knowledge_data, output_file)

        return knowledge_data

    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        print("가능한 해결방법:")
        print("1. JPype1 설치: pip install jpype1")
        print("2. tabula-py 재설치: pip install --upgrade tabula-py")
        print("3. PDF 파일 인코딩 확인")
        return None


# 실행
if __name__ == "__main__":
    pdf_file = ".\\20250630_더존비즈온.pdf"
    result = safe_process_pdf(pdf_file)

=== PDF 처리 시작 ===
처리 중 오류 발생: 'PDFToKnowledgeConverter' object has no attribute 'process_pdf'
가능한 해결방법:
1. JPype1 설치: pip install jpype1
2. tabula-py 재설치: pip install --upgrade tabula-py
3. PDF 파일 인코딩 확인


In [8]:
import PyPDF2
import pandas as pd
import re
from typing import List, Dict
import json
import os
from pathlib import Path


class PDFToKnowledgeConverter:
    def __init__(self):
        self.knowledge_base = []

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """PDF에서 텍스트 추출 (인코딩 문제 해결)"""
        text = ""
        try:
            # 바이너리 모드로 파일 열기
            with open(pdf_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        page_text = page.extract_text()
                        if page_text:
                            text += f"\n--- 페이지 {page_num + 1} ---\n"
                            text += page_text + "\n"
                    except Exception as e:
                        print(f"페이지 {page_num + 1} 텍스트 추출 실패: {e}")
                        continue
        except Exception as e:
            print(f"PDF 텍스트 추출 오류: {e}")
        return text

    def extract_tables_from_pdf(self, pdf_path: str) -> List[pd.DataFrame]:
        """PDF에서 표 추출 (인코딩 옵션 추가)"""
        try:
            import tabula

            print("tabula-py를 사용하여 표 추출 시도...")

            # 인코딩 옵션을 여러 개 시도
            encodings = ["utf-8", "cp949", "euc-kr", "cp1252", "iso-8859-1"]

            for encoding in encodings:
                try:
                    print(f"인코딩 {encoding} 시도 중...")
                    tables = tabula.read_pdf(
                        pdf_path,
                        pages="all",
                        multiple_tables=True,
                        encoding=encoding,
                        lattice=True,  # 격자 기반 테이블 추출
                        stream=True,  # 스트림 기반 테이블 추출도 시도
                    )
                    if tables:
                        print(f"✅ {encoding} 인코딩으로 {len(tables)}개 표 추출 성공")
                        return tables
                except Exception as e:
                    print(f"{encoding} 인코딩 실패: {e}")
                    continue

            print("⚠️ 모든 인코딩 방식 실패, 빈 리스트 반환")
            return []

        except ImportError:
            print("❌ tabula-py가 설치되지 않았습니다.")
            print("다음 명령어로 설치하세요:")
            print("pip install tabula-py")
            print("pip install jpype1")
            return []
        except Exception as e:
            print(f"표 추출 오류: {e}")
            return []

    def clean_text(self, text: str) -> str:
        """텍스트 정제 (한글 지원)"""
        if not text:
            return ""

        # 불필요한 공백 정리
        text = re.sub(r"\s+", " ", text)

        # 특수문자 제거하되 한글, 영문, 숫자, 기본 문장부호 유지
        text = re.sub(r"[^\w\s가-힣.,?!():\-]", " ", text)

        # 연속된 공백을 하나로
        text = re.sub(r"\s+", " ", text)

        return text.strip()

    def split_into_chunks(self, text: str, chunk_size: int = 500) -> List[str]:
        """텍스트를 청크로 분할"""
        if not text:
            return []

        # 문장 단위로 먼저 분할
        sentences = re.split(r"[.!?]\s+", text)

        chunks = []
        current_chunk = ""
        current_size = 0

        for sentence in sentences:
            words = sentence.split()
            sentence_size = len(words)

            if current_size + sentence_size <= chunk_size:
                current_chunk += sentence + ". "
                current_size += sentence_size
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
                current_size = sentence_size

        # 마지막 청크 추가
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def table_to_qa_pairs(self, table: pd.DataFrame, table_index: int) -> List[Dict]:
        """표를 Q&A 쌍으로 변환"""
        qa_pairs = []

        if table.empty:
            return qa_pairs

        try:
            # 표의 기본 정보 Q&A
            qa_pairs.append(
                {
                    "question": f"표 {table_index + 1}의 구조를 알려주세요.",
                    "answer": f"표 {table_index + 1}은 {len(table.columns)}개 열, {len(table)}개 행으로 구성되어 있습니다. 열 이름은 {', '.join(str(col) for col in table.columns)}입니다.",
                    "source": "table_structure",
                    "metadata": {
                        "table_index": table_index,
                        "columns": len(table.columns),
                        "rows": len(table),
                    },
                }
            )

            # 각 행의 데이터를 Q&A로 변환
            for row_idx, row in table.iterrows():
                if row_idx >= 10:  # 너무 많은 Q&A 생성 방지
                    break

                # 첫 번째 열을 기준으로 질문 생성
                first_col = str(table.columns[0])
                first_value = str(row.iloc[0]) if pd.notna(row.iloc[0]) else "정보 없음"

                # 해당 행의 모든 정보를 포함한 답변 생성
                answer_parts = []
                for col, value in row.items():
                    if pd.notna(value) and str(value).strip():
                        answer_parts.append(f"{col}: {value}")

                if answer_parts:
                    question = f"{first_col} '{first_value}'에 대한 정보를 알려주세요."
                    answer = (
                        f"{first_col} '{first_value}'의 정보는 다음과 같습니다. "
                        + ", ".join(answer_parts)
                        + "."
                    )

                    qa_pairs.append(
                        {
                            "question": question,
                            "answer": answer,
                            "source": "table_row",
                            "metadata": {
                                "table_index": table_index,
                                "row_index": row_idx,
                                "primary_key": first_value,
                            },
                        }
                    )

        except Exception as e:
            print(f"표 {table_index} Q&A 변환 중 오류: {e}")

        return qa_pairs

    def text_to_knowledge_structure(self, text_chunks: List[str]) -> List[Dict]:
        """텍스트 청크를 지식 구조로 변환"""
        knowledge_items = []

        for i, chunk in enumerate(text_chunks):
            if not chunk.strip():
                continue

            # 첫 문장을 제목으로 추출
            sentences = chunk.split(".")
            title = sentences[0].strip()[:100]  # 제목 길이 제한
            if not title:
                title = f"문서 섹션 {i + 1}"

            # 키워드 추출 (간단한 방법)
            words = chunk.split()
            keywords = []
            for word in words:
                if len(word) > 2 and word.isalpha():
                    keywords.append(word)
            keywords = list(set(keywords))[:10]  # 중복 제거 및 상위 10개

            knowledge_item = {
                "id": f"doc_chunk_{i}",
                "title": title,
                "content": chunk,
                "type": "document",
                "keywords": keywords,
                "metadata": {
                    "chunk_index": i,
                    "word_count": len(chunk.split()),
                    "character_count": len(chunk),
                },
            }
            knowledge_items.append(knowledge_item)

        return knowledge_items

    def process_pdf(self, pdf_path: str) -> Dict:
        """PDF 전체 처리 파이프라인"""
        result = {
            "document_knowledge": [],
            "table_qa_pairs": [],
            "summary": {},
            "metadata": {
                "source_file": pdf_path,
                "processing_timestamp": pd.Timestamp.now().isoformat(),
            },
        }

        print(f"📄 PDF 처리 시작: {pdf_path}")

        # 1. 텍스트 추출 및 처리
        print("📝 텍스트 추출 중...")
        raw_text = self.extract_text_from_pdf(pdf_path)

        if raw_text:
            cleaned_text = self.clean_text(raw_text)
            text_chunks = self.split_into_chunks(cleaned_text, chunk_size=300)
            print(f"✅ {len(text_chunks)}개의 텍스트 청크 생성")

            # 2. 텍스트를 지식 구조로 변환
            print("🔄 텍스트 지식 구조 생성 중...")
            result["document_knowledge"] = self.text_to_knowledge_structure(text_chunks)
        else:
            print("⚠️ 추출된 텍스트가 없습니다.")

        # 3. 표 추출 및 Q&A 쌍 생성
        print("📊 표 추출 및 Q&A 생성 중...")
        tables = self.extract_tables_from_pdf(pdf_path)

        for table_idx, table in enumerate(tables):
            if not table.empty:
                print(f"📋 표 {table_idx + 1} 처리 중... (크기: {table.shape})")
                qa_pairs = self.table_to_qa_pairs(table, table_idx)
                result["table_qa_pairs"].extend(qa_pairs)

        # 4. 요약 정보
        result["summary"] = {
            "total_text_chunks": len(result["document_knowledge"]),
            "total_tables": len(tables),
            "total_qa_pairs": len(result["table_qa_pairs"]),
            "total_knowledge_items": len(result["document_knowledge"]),
            "original_text_length": len(raw_text) if raw_text else 0,
        }

        print("✅ PDF 처리 완료!")
        return result

    def save_knowledge_base(self, knowledge_data: Dict, output_path: str):
        """지식 베이스를 JSON 파일로 저장"""
        try:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(knowledge_data, f, ensure_ascii=False, indent=2)
            print(f"💾 지식 베이스가 {output_path}에 저장되었습니다.")
        except Exception as e:
            print(f"❌ 저장 실패: {e}")


def safe_process_pdf(pdf_path: str):
    """안전한 PDF 처리 함수"""

    # Path 객체로 경로 처리
    path = Path(pdf_path)

    print(f"🔍 파일 경로 확인: {path}")
    print(f"📁 절대 경로: {path.absolute()}")

    if not path.exists():
        print(f"❌ 파일을 찾을 수 없습니다: {pdf_path}")

        # 현재 디렉토리에서 PDF 파일 검색
        print("\n🔎 현재 디렉토리에서 PDF 파일 검색:")
        for pdf_file in Path(".").rglob("*.pdf"):
            print(f"📄 발견: {pdf_file}")

        return None

    try:
        # 파일 크기 확인
        file_size = path.stat().st_size
        print(f"📊 파일 크기: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")

        converter = PDFToKnowledgeConverter()

        print("\n" + "=" * 50)
        print("🚀 PDF 지식 구조 변환 시작")
        print("=" * 50)

        knowledge_data = converter.process_pdf(str(path))

        print("\n" + "=" * 50)
        print("📊 처리 결과 요약")
        print("=" * 50)
        summary = knowledge_data["summary"]
        print(f"📝 텍스트 청크: {summary['total_text_chunks']:,}개")
        print(f"📊 추출된 표: {summary['total_tables']:,}개")
        print(f"❓ 생성된 Q&A 쌍: {summary['total_qa_pairs']:,}개")
        print(f"📚 총 지식 아이템: {summary['total_knowledge_items']:,}개")
        print(f"📄 원본 텍스트 길이: {summary['original_text_length']:,} 문자")

        # 결과 저장
        output_file = path.stem + "_knowledge_base.json"
        converter.save_knowledge_base(knowledge_data, output_file)

        # 샘플 출력
        print("\n" + "=" * 50)
        print("📋 샘플 결과 미리보기")
        print("=" * 50)

        if knowledge_data["document_knowledge"]:
            print("\n📝 문서 지식 샘플:")
            sample = knowledge_data["document_knowledge"][0]
            print(f"제목: {sample['title']}")
            print(f"내용: {sample['content'][:200]}...")
            print(f"키워드: {sample['keywords'][:5]}")

        if knowledge_data["table_qa_pairs"]:
            print("\n📊 표 Q&A 샘플:")
            sample = knowledge_data["table_qa_pairs"][0]
            print(f"질문: {sample['question']}")
            print(f"답변: {sample['answer'][:200]}...")

        return knowledge_data

    except Exception as e:
        print(f"\n❌ 처리 중 오류 발생: {e}")
        print("\n🔧 해결 방법:")
        print("1. 필수 패키지 설치:")
        print("   pip install jpype1")
        print("   pip install tabula-py")
        print("   pip install PyPDF2")
        print("   pip install pandas")
        print("2. PDF 파일 인코딩 및 권한 확인")
        print("3. Java 설치 확인 (tabula-py 필요)")
        return None


# 메인 실행 부분
if __name__ == "__main__":
    # 더존비즈온 PDF 파일 처리
    pdf_file = ".\\20250630_더존비즈온.pdf"

    print("🤖 PDF to 챗봇 지식 구조 변환기")
    print("=" * 50)

    result = safe_process_pdf(pdf_file)

    if result:
        print("\n🎉 처리가 성공적으로 완료되었습니다!")
        print("생성된 지식 베이스를 챗봇 학습에 활용할 수 있습니다.")
    else:
        print("\n😞 처리에 실패했습니다. 오류 메시지를 확인해주세요.")

🤖 PDF to 챗봇 지식 구조 변환기
🔍 파일 경로 확인: 20250630_더존비즈온.pdf
📁 절대 경로: c:\workdir\github-space-cap\GPT-study-blog\langchain_project\lee\20250630_더존비즈온.pdf
📊 파일 크기: 1,741,038 bytes (1.66 MB)

🚀 PDF 지식 구조 변환 시작
📄 PDF 처리 시작: 20250630_더존비즈온.pdf
📝 텍스트 추출 중...
✅ 7개의 텍스트 청크 생성
🔄 텍스트 지식 구조 생성 중...
📊 표 추출 및 Q&A 생성 중...
tabula-py를 사용하여 표 추출 시도...
인코딩 utf-8 시도 중...
utf-8 인코딩 실패: 'utf-8' codec can't decode byte 0xb8 in position 3153: invalid start byte
인코딩 cp949 시도 중...
✅ cp949 인코딩으로 36개 표 추출 성공
📋 표 2 처리 중... (크기: (1, 1))
📋 표 5 처리 중... (크기: (1, 1))
📋 표 10 처리 중... (크기: (9, 24))
📋 표 20 처리 중... (크기: (26, 3))
📋 표 25 처리 중... (크기: (10, 13))
📋 표 26 처리 중... (크기: (1, 2))
📋 표 27 처리 중... (크기: (2, 3))
📋 표 29 처리 중... (크기: (1, 2))
📋 표 30 처리 중... (크기: (1, 2))
📋 표 31 처리 중... (크기: (1, 3))
📋 표 32 처리 중... (크기: (2, 3))
📋 표 34 처리 중... (크기: (1, 1))
📋 표 35 처리 중... (크기: (1, 2))
✅ PDF 처리 완료!

📊 처리 결과 요약
📝 텍스트 청크: 7개
📊 추출된 표: 36개
❓ 생성된 Q&A 쌍: 32개
📚 총 지식 아이템: 7개
📄 원본 텍스트 길이: 10,129 문자
💾 지식 베이스가 20250630_더존비즈온_knowledge_base.json에 저장